In [ ]:
import arviz as az
import model
import os
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import random
import m2l

In [ ]:
alf_home = os.environ.get('ALF_HOME')
jalf_home = os.environ.get('JALF_HOME')

#read in data to fit
filename = 'NGC2695_KCWI_3ring_001'
indata_file = jalf_home+'indata/'+filename
mo = model.model(indata_file)

In [ ]:

posterior_samples = np.load(jalf_home+'results/'+filename+'.npz')

param_list = ['age','Z','imf1','velz','sigma','nah','cah','feh','ch','nh',
              'ah','tih','mgh','sih','mnh','bah','nih','coh','euh','srh','kh','vh','cuh','teff',
              'loghot','hotteff','logm7g',
              'age_young','log_frac_young']

median_params = []
n_to_plot = 100
n_ind = random.sample(range(len(posterior_samples['age'])),100)
n_params = []
for pname in param_list:
    samples = posterior_samples[pname]
    if pname == 'age':
        median_params.append(np.median(np.log10(samples)))
        n_params.append(np.log10(samples[n_ind]))
    elif pname == 'imf1':
        median_params.append(np.median(np.log10(samples)))
        median_params.append(np.median(np.log10(samples)))
        n_params.append(np.log10(samples[n_ind]))
        n_params.append(np.log10(samples[n_ind]))

    elif (pname == 'velz') or (pname == 'sigma') or (pname == 'teff'):
        median_params.append(np.median(samples) * 100)      
        n_params.append(samples[n_ind]*100)
    elif (pname != 'df'):
        median_params.append(np.median(samples))
        n_params.append(samples[n_ind])
n_params=np.array(n_params).T

In [ ]:
n_params.shape

In [ ]:
lam, flux = mo.model_flux_total(median_params)
m2l.getm2l(lam,flux,median_params[0],median_params[1],median_params[2],median_params[2])

In [ ]:
i = 3

wl_d_region, flux_d_region, dflux_d_region, flux_m_region, flux_mn_region = mo.model_flux_regions(median_params)
plt.plot(wl_d_region[i],flux_d_region[i],color='k')
plt.plot(wl_d_region[i],flux_mn_region[i],color='r',alpha=0.5)
for j in range(n_to_plot):
    wl_d_region, flux_d_region, dflux_d_region, flux_m_region, flux_mn_region = mo.model_flux_regions(n_params[j])
    plt.plot(wl_d_region[i],flux_mn_region[i],color='orange',alpha=0.05)
#plt.xlim(8000,8750)

In [ ]:
idata = az.from_netcdf(jalf_home+'results/'+filename+'.nc')

az.plot_pair(idata, 
             var_names=['age','Z','imf1','velz','sigma'],
             kind='kde',
             marginals=True)
plt.show()

In [ ]:
alf_home = os.environ.get('ALF_HOME')
jalf_home = os.environ.get('JALF_HOME')

param_list = ['age','Z','imf1','imf2','velz','sigma','nah','cah','feh','ch','nh',
                'ah','tih','mgh','sih','mnh','bah','nih','coh','euh','srh','kh','vh','cuh','teff',
                'loghot','hotteff','logm7g']

glist = ['ESO325-G004','NGC1277','NGC1407','NGC1600','NGC2695','GCcombine']
fit_values = []
for gname in glist:
    if gname == 'GCcombine':
        filename = gname+'_r8vd100_001'
    else:
        filename = gname+'_supercoarse_001'
    indata_file = jalf_home+'indata/'+filename
    mo = model.model(indata_file,ang_per_poly_degree=200)
    posterior_samples = np.load(jalf_home+'results/'+filename+'.npz')

    median_params = []
    n_to_plot = 100
    n_ind = random.sample(range(len(posterior_samples['age'])),100)
    n_params = []
    for pname in param_list:
        samples = posterior_samples[pname]
        if pname == 'age':
            median_params.append(np.median(np.log10(samples)))
            n_params.append(np.log10(samples[n_ind]))

        elif (pname == 'velz') or (pname == 'sigma') or (pname == 'teff'):
            median_params.append(np.median(samples) * 100)      
            n_params.append(samples[n_ind]*100)
        elif (pname != 'df'):
            median_params.append(np.median(samples))
            n_params.append(samples[n_ind])
    n_params=np.array(n_params).T

    i = 4

    wl_d_region, flux_d_region, dflux_d_region, flux_m_region, flux_mn_region = mo.model_flux_regions(median_params)
    plt.plot(wl_d_region[i],flux_d_region[i],color='k')
    plt.plot(wl_d_region[i],flux_mn_region[i],color='r',alpha=0.5)
    for j in range(n_to_plot):
        wl_d_region, flux_d_region, dflux_d_region, flux_m_region, flux_mn_region = mo.model_flux_regions(n_params[j])
        plt.plot(wl_d_region[i],flux_mn_region[i],color='orange',alpha=0.05)
    plt.title(gname)
    plt.show()

    velz_percentile = np.percentile(posterior_samples['velz'],(16,50,84))*100
    sigma_percentile = np.percentile(posterior_samples['sigma'],(16,50,84))*100
    if gname != 'GCcombine':
        sigma_percentile = np.sqrt(sigma_percentile**2 + 100**2)

    fit_values.append(np.append(velz_percentile,sigma_percentile))



In [ ]:
combined = np.column_stack((glist,np.vstack(fit_values).astype(str)))
header = 'name, velz_16, velz_50, velz_84, sigma_16, sigma_50, sigma_84'
np.savetxt('vel_fit.csv',combined,fmt='%s', delimiter=',',header=header)